In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt, generate_krylov_space_othogonal_against
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [2]:
#load integrals
#with h5py.File("data_water/eri_water.hdf5", "r") as f:
with h5py.File("/work_fast/ge49cag/code_datas/hchain/NH_10/integral.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/hchain/NH_10/u.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

#ref energy got from DMRG
e_1st_ex = -12.36246700876214
e_ground = -12.407196480372146

10
(10, 27)
(27, 27)
rl errV: 4.598965179239031e-05
abs errV: 0.00014647113920761016
errt: 0.0007926989428724057
errh: 0.00010479314802464695
errht: 0.00025419491299631246


These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [3]:
# #load Hamiltonian generated by above coefficients
#H_correct = scipy.io.mmread('/.mtx').tocsr()
# e1, v1 = sparse.linalg.eigsh(H_correct, which = 'SA', k = 1)
# e_ground = e1

# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)

Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [4]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]


We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [5]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 70, 108, 162, 232, 162, 108, 70, 16, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [6]:
#temp1 = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0])
#initial = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0])

initial = generate_single_state(10, [3, 3, 3, 3, 1, 2, 0, 0, 0, 0])

#hartree_state = add_mps(add_mps(add_mps(temp1, temp2), temp3),temp4)
# hartree_state = add_mps(temp1, temp2)
# hartree_state.orthonormalize('left')
# hartree_state.orthonormalize('right')

In [7]:
filename = f"/work_fast/ge49cag/code_datas" + f"/H10_ground_state.pkl"
with open(filename, 'rb') as file:
    H10_ground = pickle.load(file)

ptn.operation.operator_average(H10_ground, mpo_ref) - e_ground

(9.296700387295687e-07+0j)

In [8]:
ptn.operation.vdot(H10_ground, initial)

0j

In [ ]:
#ptn.operation.operator_average(initial, mpo_ref) - e_1st_ex

(6.294095970736529+0j)

In [ ]:
N_Krylov_1 = 30
psi_original_1 = copy.deepcopy(initial)
max_bond_Krylov_1 = 250
trunc_tol = 0
foldername_1 = f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex"
#generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)
#generate_krylov_space_othogonal_against(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1, copy.deepcopy(H10_ground))

[1, 4, 11, 22, 37, 82, 37, 22, 11, 4, 1]
2


KeyboardInterrupt: 

Make use of method proposed in https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.205119 to improve the orthogonality of Krylov vectors. 

In [27]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)


/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [28]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_1st_ex, mpo_ref)

(4.582640627839472+0j)
(3.2229389444958993+0j)
(2.129976311413829+0j)
(1.1632848382456373+0j)
(0.5430210502644943+0j)
(0.3116758104748296+0j)


Restart:

In [29]:
N_Krylov_2 = 30
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 250
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex_restart"
#generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)
generate_krylov_space_othogonal_against(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2, copy.deepcopy(H10_ground))

[1, 4, 16, 64, 246, 250, 246, 64, 16, 4, 1]
2
[1, 4, 16, 64, 245, 250, 245, 64, 16, 4, 1]
(0.9999999999999984+0j)
(1.3713266633352816e-12+0j)
3
[1, 4, 16, 64, 243, 250, 243, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-8.30533558593416e-12+0j)
4
[1, 4, 16, 64, 241, 250, 241, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-1.8011356295311032e-12+0j)
5
[1, 4, 16, 64, 240, 250, 240, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-1.8869263790355362e-10+0j)
6
[1, 4, 16, 64, 245, 250, 245, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-3.953404270617966e-11+0j)
7
[1, 4, 16, 64, 246, 250, 246, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-9.621081709099144e-11+0j)
8
[1, 4, 16, 64, 247, 250, 247, 64, 16, 4, 1]
(0.9999999999999984+0j)
(9.333053774263078e-11+0j)
9
[1, 4, 16, 64, 247, 250, 247, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-2.3827759210170996e-11+0j)
10
[1, 4, 16, 64, 247, 250, 247, 64, 16, 4, 1]
(0.9999999999999984+0j)
(1.287284168149938e-11+0j)
11
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(6

In [30]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)


In [31]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_1st_ex, mpo_ref)

(0.0675043880020656+0j)
(0.011417985326902524+0j)
(0.003954528640703003+0j)
(0.0006487794500724675+0j)
(0.00010395878647706525+0j)
(5.4098321257356474e-05+0j)


Restart:

In [32]:
N_Krylov_3 = 20
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 250
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex_restart_2"
#generate_krylov_space_in_disk(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3)
generate_krylov_space_othogonal_against(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3, copy.deepcopy(H10_ground))

#the initial state is now a approximation of 1st-ex state, which contains more ground state component than others
#we can also rothogonalize to ground state when re-orthogonalizing

[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
2
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(4.547045023998997e-06+0j)
3
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-3.3197219906980213e-06+0j)
4
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(1.093098293034056e-06+0j)
5
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-8.574888826537452e-06+0j)
6
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(1.385924626430557e-05+0j)
7
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(2.0117181880295598e-05+0j)
8
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-1.876550123568882e-05+0j)
9
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(5.296305987528838e-06+0j)
10
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-6.998558242481134e-07+0j)
11
[1, 4, 16, 64, 249, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(9.37

In [33]:
H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)

In [34]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_1st_ex, mpo_ref)

(2.521489164486468e-05+0j)
(1.8548244623772803e-05+0j)
(1.700892502221052e-05+0j)
(1.6818705345755802e-05+0j)


Restart:

In [40]:
N_Krylov_4 = 10
#psi_original_4 = copy.deepcopy(v_ritz_3)
max_bond_Krylov_4 = 250
trunc_tol = 0
foldername_4 = f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex_restart_3"
#generate_krylov_space_othogonal_against(N_Krylov_4, H_mu_nu_list_spin_layer, psi_original_4, max_bond_Krylov_4, trunc_tol, r_thc, foldername_4, copy.deepcopy(H10_ground))


In [41]:
H_reduced_non_rotho_4 = generate_reduced_H_non_ortho(N_Krylov_4, foldername_4, mpo_ref)
coeff_4 = coeff_gram_schmidt(N_Krylov_4, foldername_4)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_4 = np.einsum('ik, kl, jl -> ij', coeff_4.conj(), H_reduced_non_rotho_4, coeff_4)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [42]:
e_ritz_4, v_ritz_4 = solve_ritz(foldername_4, H_reduced_4, N_Krylov_4, coeff_4, max_bond_Krylov_4, e_1st_ex, mpo_ref)

(6.089186287638881e-06+0j)
(5.995346358034226e-06+0j)


In [ ]:
# (6.089186287638881e-06+0j)
# (5.995346358034226e-06+0j)
# (5.9439894055657305e-06+0j)
# (5.904399229095247e-06+0j)

In [39]:
#maybe change M=200 to see whether it can still get correct state 
#and picture both M=250 and M=200 to say good result can still be obtain

In [43]:
N_Krylov_5 = 10
psi_original_5 = copy.deepcopy(v_ritz_4)
max_bond_Krylov_5 = 250
trunc_tol = 0
foldername_5 = f"/work_fast/ge49cag/code_datas/Krylov_H10_1st_ex_restart_4"
generate_krylov_space_othogonal_against(N_Krylov_5, H_mu_nu_list_spin_layer, psi_original_5, max_bond_Krylov_5, trunc_tol, r_thc, foldername_5, copy.deepcopy(H10_ground))


[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
2
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-0.00016798497723482554+0j)
3
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-1.8664324232402912e-05+0j)
4
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(1.2597752175949606e-05+0j)
5
[1, 4, 16, 64, 249, 250, 249, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-4.342519890856988e-06+0j)
6
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(5.450885861721377e-06+0j)
7
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-2.827380740297314e-06+0j)
8
[1, 4, 16, 64, 249, 250, 249, 64, 16, 4, 1]
(0.9999999999999984+0j)
(3.698823513240135e-06+0j)
9
[1, 4, 16, 64, 250, 250, 250, 64, 16, 4, 1]
(0.9999999999999984+0j)
(-4.314108963371366e-06+0j)


In [44]:
H_reduced_non_rotho_5 = generate_reduced_H_non_ortho(N_Krylov_5, foldername_5, mpo_ref)
coeff_5 = coeff_gram_schmidt(N_Krylov_5, foldername_5)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_5 = np.einsum('ik, kl, jl -> ij', coeff_5.conj(), H_reduced_non_rotho_5, coeff_5)

In [45]:
e_ritz_5, v_ritz_5 = solve_ritz(foldername_5, H_reduced_5, N_Krylov_5, coeff_5, max_bond_Krylov_5, e_1st_ex, mpo_ref)

(2.6838109885574113e-06+0j)
(2.581122449996087e-06+0j)


In [46]:
e_final, v_final = np.linalg.eigh(H_reduced_5)
print(e_final[0] - e_1st_ex)

2.6065444114209413e-06


In [ ]:
error_list_ground = [
    4.9895638575465835,
    2.9754418254647845,
    1.8842586919638862,
    1.118923977507352,
    0.5891514410222225,
    0.36575387369147805,
    0.06698301097829074,
    0.0026002560049107615,
    0.0003776699617112911,
    3.4391775949771386e-05,
    1.728864507200001e-05,
    1.5936947953321123e-05,
    3.2456567744532094e-06,
    3.0369881329761483e-06,
    3.032986882089972e-06,
    3.0320862745014665e-06,
    9.319979259458933e-07,
    9.30569894919131e-07,
    9.302608212635732e-07,
    9.297268608321474e-07
]


In [1]:
error_list_1st_ex = [
    4.582640627839472,
    3.2229389444958993,
    2.129976311413829,
    1.1632848382456373,
    0.5430210502644943,
    0.3116758104748296,
    0.0675043880020656,
    0.011417985326902524,
    0.003954528640703003,
    0.0006487794500724675,
    0.00010395878647706525,
    5.4098321257356474e-05,
    2.521489164486468e-05,
    1.8548244623772803e-05,
    1.700892502221052e-05,
    1.6818705345755802e-05,
    6.089186287638881e-06,
    5.995346358034226e-06,
    2.6838109885574113e-06,
    2.581122449996087e-06
]
